In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1, thousands=',')

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

def answer_one():
    df.sort_values('Gold', ascending=False).iloc[0].name

    df['Diff'] = df['Gold'] - df['Gold.1']
    return df.sort_values('Diff', ascending=False).iloc[0].name

def answer_two():
    df['Diff'] = df['Gold'].astype(int) - df['Gold.1'].astype(int)
    return df.sort_values('Diff', ascending=False).iloc[0].name


def answer_three():
    only_gold = df
    only_gold['Diff2'] = (df['Gold'] - df['Gold.1'])/(df['Gold'] + df['Gold.1']).where((df['Gold'] > 0) & (df['Gold.1'] > 0))
    return only_gold.sort_values('Diff2', ascending=False).iloc[0].name

def answer_four():
    df['Gold.2'].replace(',', '')
    points = df['Gold.2'].astype(int)*3 + df['Silver.2'].astype(int)*2 + df['Bronze.2'].astype(int)
    points.rename('Points')
    return points

census_df = pd.read_csv('census.csv')
census_df.head()

def answer_five():
    df=census_df[census_df['SUMLEV'] == 50]
    return df.iloc[census_df['STATE'].value_counts().idxmax()]['STNAME']

def answer_six():
    df = census_df[census_df['SUMLEV'] == 50]
    by_county = df.groupby('STNAME')['CENSUS2010POP'].nlargest(3).reset_index().groupby('STNAME').sum()
    return by_county.sort_values(by='CENSUS2010POP', ascending=False)[:3].index.tolist()

def answer_seven():
    df = census_df[census_df['SUMLEV'] == 50].set_index('CTYNAME')
    last_five_years = df[['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']]
    delta = last_five_years.max(axis=1) - last_five_years.min(axis=1)
    return delta.sort_values(ascending=False).reset_index().iloc[0]['CTYNAME']

def answer_eight():
    df = census_df[census_df['SUMLEV'] == 50]
    region = df['REGION']
    p2014 = df['POPESTIMATE2014']
    p2015 = df['POPESTIMATE2015']

    filtered = df[((region == 1) | (region == 2)) & (df['CTYNAME'].str.startswith('Washington')) & (p2015 > p2014)]
    return filtered[['STNAME', 'CTYNAME']].sort_index()

OSError: File b'olympics.csv' does not exist